In [1]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath


### Download & Process Glove. Ignore if mod file exists

In [ ]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip

model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt')
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

generate_norm_embedding(model_glove,'glove_norm_300.mod')


### Load processed glove

In [2]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('/Users/stjepankusenic/POLAR_WEBE/notebooks/glove_norm_300.mod',binary=True) # ss


In [3]:
current_model = model_gn


### Load and process antonyms. Ignore if final_antonym_list exists.

In [ ]:
list_antonym = []

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))

with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

### Load final_antonym_list as antonym list

In [ ]:
import pandas as pd

list_antonym = pd.read_pickle(r'final_antonym_list')

In [12]:
list_new= [('product', 'service'),
('essential', 'luxury'),
('technical', 'natural'),
('renewable', 'nonrenewable'),
('advertising', 'rumour'), 
('lease', 'sell'), 
('tangible', 'intangible'), 
('demand', 'supply'),
('wfh', 'wfo'),
('child', 'childless'),
('remote', 'physical'),
('salary', 'goodies'),
('store', 'online'),
('details', 'outlines'),
('stakeholders', 'spectators'),
('isolating', 'social'),
('goal', 'task'),
('employees', 'consultant'),
('cost', 'revenue'),
('seasonal', 'temporary'),
('alliance', 'proprietorship'),
('loss', 'profit'),
('integrity', 'corruption'),
('international', 'local'),
('corporate', 'individual'),
('order', 'disorder'),
('solution', 'problem'),
('manager', 'worker'),
('diversity', 'uniformity'),
('public', 'private'),
('strategic', 'impulsive'),
('innovator', 'follower'),
('bankruptcy', 'prosperity'),
('growth', 'decline'),
('sustainable', 'unsustainable'),
('family', 'work'),
('criminal', 'rightful'),
('financial', 'artisanal'),
('supplier', 'purchaser'),
('commitment', 'rejection'),
('professional', 'amateur'),
('independent', 'dependent'),
('digital', 'analogue'),
('marketing', 'secret'),
('secure', 'risky'),
('longterm', 'shortterm'), 
('responsible', 'neglect'), 
('ethical', 'unethical'), 
('beneficial', 'harmful'), 
('diversity', 'uniformity'), 
('trust', 'mistrust'), 
('teamwork', 'individualism'), 
('opportunity', 'threat'), 
('innovative', 'traditional'), 
('flexible', 'rigid'), 
('ambiguity', 'clarity'), 
('feminine', 'masculine'), 
('globally', 'locally'), 
('insiders', 'outsiders'), 
('foreigners', 'natives'), 
('minorities', 'majority'),
('transparency', 'obscurity'),
('discrimination', 'impartial'),
('credible', 'deceptive'),
('environment', 'pollution'),
('pressure', 'relax'),
('growth', 'decline'),
('satisfied', 'unsatisfied'),
('diplomatic', 'undiplomatic'),
('motivate', 'demotivate'),
('communicative', 'uncommunicative'),
('connected', 'disconnected'),
('autonomous', 'micromanagement'),
('nurture', 'neglect'),
('progressive', 'conservative'),
('rewarding', 'unrewarding'),
('bias', 'unbias'),
('challenge', 'obscurity'),
('collaborated', 'siloed'),
('outdated', 'modern'),
('effortless', 'demanding'),
('economic', 'overpriced'),
('widespread', 'local'),
('freedom', 'captive'),
('consistent', 'inconsistent')]

In [13]:
list_new= list(dict.fromkeys(list_new).keys())

similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_new):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_list.append((each_key, listofTuples[0][0]))
print(len(final_list))

list_antonym = final_list


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/2013486443.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_pair in tqdm(list_new):


  0%|          | 0/83 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/2013486443.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_key in tqdm(similarity_matrix):


  0%|          | 0/82 [00:00<?, ?it/s]

/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/2013486443.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index_counter, each_key in enumerate(tqdm(all_similarity)):


  0%|          | 0/82 [00:00<?, ?it/s]

82


### Decide on the size of the antonym vector


In [14]:
num_antonym = 82

## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(82, 300)


### Subset Dimension Selection Method


In [15]:
import random
import scipy

random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = scipy.spatial.distance.cdist(np.array(antonymy_vector),np.array(antonymy_vector),'cosine')
dimension_similarity_matrix = abs(1-dimension_similarity_matrix)
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
    return final_position_index

## Select POLAR dimensions

#### Generate the RANDOM DIMENSION Order


In [16]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

82


#### Generate the ORTHOGONAL DIMENSION Order


In [17]:
orthogonal_antonymy_vector =np.array(select_subset_dimension(antonymy_vector, num_antonym))  
print(orthogonal_antonymy_vector.shape)

working list is ready, shape (82, 300)


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/1541790553.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for test_count in tqdm(range(num_dim-1)):


  0%|          | 0/81 [00:00<?, ?it/s]

(82,)


#### Generate the MAXIMUM VARIANCE DIMENSION Order


In [20]:
import torch

embedding_size = antonymy_vector.shape[0]   
print('The embedding size is', embedding_size)


variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vector))
variance_antonymy_vector_inverse = torch.tensor(variance_antonymy_vector_inverse)

embedding_matrix = []

current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))

The embedding size is 82


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/2849266527.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))


In [21]:
var_list = [None for x in range(20)] # variance for each antonym in each batch

for i in range(19):  # the first 19 batches, each of size 100k
  temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,100000*i:100000*i+100000])
  temp_var_mean = torch.var(temp, axis = 1)
  var_list[i] = temp_var_mean.numpy()
  del temp

temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,1900000:])
temp_var_mean = torch.var(temp, axis = 1)
var_list[19] = temp_var_mean.numpy()
del temp

In [22]:
# using lazy approach. assume each batch is independent and the overall variance is the average variance over all batches

variance_list = np.mean(np.array(var_list),axis = 0)

variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]


### Begin POLAR embedding

In [23]:
import pandas as pd

company = pd.read_csv('/Users/stjepankusenic/POLAR_WEBE/data/interim/glove-ticker-fortune1000-us.csv')
ticker_list = company['Ticker'].str.lower() # a list of tickers

In [24]:
ticker_word_embedding = dict()
for ticker in ticker_list:    # dictionary of ticker: glove embedding
  ticker_word_embedding[ticker] = current_model[ticker]

In [25]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

In [26]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

Select the number of POLAR demensions and the method of antonym selection

In [27]:
import subprocess
dim_size = 82 # Number of POLAR dimenions
antonym_vector_method = orthogonal_antonymy_vector 
# random_antonym_vector or orthogonal_antonymy_vector or variance_antonymy_vector 
antonym_500 = [list_antonym[x] for x in antonym_vector_method[:82]]


In [28]:
# create POLAR embedding of tickers
ticker_new_embedding = generate_embedding_path(ticker_word_embedding, 'test_run',True ,antonym_vector_method,82) 


New model size is 892 82


/var/folders/xy/yfm2dhtj3jvfy85hqy16wmk00000gn/T/ipykernel_59796/2921753199.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for each_word in tqdm(current_model):


  0%|          | 0/892 [00:00<?, ?it/s]

#### View top embeddings

In [29]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

In [30]:
# amazon 
amzn_polar = make_polar_dict('amzn', antonym_500, ticker_new_embedding, True)


('spectators', 'stakeholders') -0.16217703 

('foreigners', 'natives') -0.16648787 

('analogue', 'digital') -0.1780018 

('loss', 'profit') -0.17841296 

('conservative', 'progressive') 0.19495878 

('demand', 'supply') 0.19550358 

('insiders', 'outsiders') 0.19992629 

('artisanal', 'financial') -0.24137904 

('economic', 'overpriced') -0.25286204 

('beneficial', 'harmful') 0.28406265 



In [31]:
# walmart
wmt_polar = make_polar_dict('wmt', antonym_500, ticker_new_embedding, True)


('modern', 'outdated') -0.19774586 

('feminine', 'masculine') 0.19941388 

('diplomatic', 'undiplomatic') -0.20358516 

('nonrenewable', 'renewable') 0.21047162 

('online', 'store') -0.21100266 

('consistent', 'inconsistent') 0.22974993 

('globally', 'locally') 0.24147882 

('beneficial', 'harmful') 0.25350198 

('consultant', 'employees') -0.27024293 

('decline', 'growth') -0.27411142 



In [32]:
# facebook
fb_polar = make_polar_dict('fb', antonym_500, ticker_new_embedding, True)


('manager', 'worker') 0.18059273 

('problem', 'solution') 0.18642288 

('intangible', 'tangible') -0.1915821 

('details', 'outlines') 0.19724244 

('analogue', 'digital') -0.21507092 

('economic', 'overpriced') -0.25961307 

('isolating', 'social') -0.26350588 

('seasonal', 'temporary') -0.26964805 

('consistent', 'inconsistent') 0.295854 

('follower', 'innovator') 0.32891008 



In [33]:
# wells fargo
wfc_polar = make_polar_dict('wfc', antonym_500, ticker_new_embedding, True) 


('wfh', 'wfo') 0.18201385 

('connected', 'disconnected') -0.18227455 

('bankruptcy', 'prosperity') 0.18380293 

('private', 'public') -0.1954872 

('globally', 'locally') 0.20538466 

('beneficial', 'harmful') 0.20755509 

('ambiguity', 'clarity') -0.22157072 

('decline', 'growth') -0.22775714 

('consistent', 'inconsistent') 0.32833746 

('demand', 'supply') -0.37309232 

